# Entrenamiendo del Merged

In [64]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
import os
import json

import random as python_random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras import layers, models

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

from keras_tuner import HyperModel, HyperParameters
from keras_tuner.tuners import Hyperband

from funcionesComunes import *

# Creación de los archivos

In [ ]:
for archivo in os.listdir("RCPMerged/"):

    if archivo != "totalMerged.csv":

        # Archivo unido de las 14 especies
        data_merged_train = pd.DataFrame()
        data_merged_val = pd.DataFrame()
        data_merged_test = pd.DataFrame()

        for subarchivo in os.listdir("RCPMerged/"):

            if subarchivo != archivo and subarchivo != "totalMerged.csv":

                # Cargamos el archivo
                df = pd.read_csv(f"RCPMerged/{subarchivo}")

                # Repetimos mismo proceso de siempre
                df = df[~df["nametag"].str.startswith("INDI005")]

                # Codificamos, normalización y split de datos
                df = codification(df)
                print(df.shape)

                df, valorNormalizacion = individualNormalization(df)
                print(f"SE HA NORMALIZADO EL ARCHIVO: {subarchivo}")

                # Split de cada df
                train_data, val_data, test_data = split_population_individuals(df, train_pct=0.80, val_pct_in_train=0.20, details=False)
                train_data.shape, val_data.shape, test_data.shape

                # Unimos cada spli de cada sub especie en el df_merged que corresponde
                data_merged_train = pd.concat([data_merged_train, train_data])
                data_merged_val = pd.concat([data_merged_val, val_data])
                data_merged_test = pd.concat([data_merged_test, test_data])

            
        # Obtenemos X e y para los datasets de train, val y test para cada subespecie
        print(f'NUMERO DE INDIVIDUOS: {len(data_merged_train["nametag_encoded"].unique())}')
        print(f'NUMERO DE INDIVIDUOS: {len(data_merged_val["nametag_encoded"].unique())}')
        print(f'NUMERO DE INDIVIDUOS: {len(data_merged_test["nametag_encoded"].unique())}')
        WINDOWS_SIZE = 3
        X_train, y_train = df_to_X_y_ind_3(data_merged_train, WINDOWS_SIZE)
        X_val, y_val = df_to_X_y_ind_3(data_merged_val, WINDOWS_SIZE)
        X_test, y_test = df_to_X_y_ind_3(data_merged_test, WINDOWS_SIZE)
        print(X_train.shape, X_test.shape, X_val.shape)

        # Guardamos los nuevos df
        np.savez(f'RCPMergedTransferMerged/{archivo.split("_merged")[0]}_train.npz', X_train=X_train, y_train=y_train, valorNormalizacion = valorNormalizacion)
        np.savez(f'RCPMergedTransferMerged/{archivo.split("_merged")[0]}_val.npz', X_val=X_val, y_val=y_val, valorNormalizacion = valorNormalizacion)
        np.savez(f'RCPMergedTransferMerged/{archivo.split("_merged")[0]}_test.npz', X_test=X_test, y_test=y_test, valorNormalizacion = valorNormalizacion)

# Ajuste hiperparametros para cada uno

In [65]:
class LSTMHyperModel(HyperModel):
    
    def __init__(self, input_shape, num_lstm_layers_input=3, num_lstm_layers_after_input=3):
        self.input_shape = input_shape
        self.num_lstm_layers_input = num_lstm_layers_input
        self.num_lstm_layers_after_input = num_lstm_layers_after_input
    
    def build(self, hp):
        
        # Podemos ver como "hp.Int" permite ir modificando los parámetros del modelo con INT
        # Con "hp.Float" lo hace pero para valores decimales 
        # IMPORTANTE!!! Nunca se devuelve el return_sequences si la LSTM es la última capa
        # IMPORTANTE!!! Si estamos aplicando capas LSTM despues Dropout y luego LSTM de nuevo, debemos de poner en todas las primeras capas LSTM return_sequences=True !!!IMPORTANTE
        
        model = Sequential()
        
        # Número de capas LSTM
        num_lstm_layers = hp.Int('num_lstm_layers', min_value=1, max_value=self.num_lstm_layers_input, default = self.num_lstm_layers_input)

        for i in range(num_lstm_layers):
            # Añadiendo capas LSTM
            # Solo la primera capa necesita input_shape
            if i == 0:
                
                # Se tiene una capa LSTM que empieza con 32 unidades y va aumentando de 32 en 32 hasta 128
                model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                               input_shape=self.input_shape,
                               return_sequences=True,  # True si hay más de una capa LSTM
                               use_bias=True))
            else:
                model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                               return_sequences=True,  # True para todas excepto la última capa LSTM
                               use_bias=True))
        
        # Se modifica la capa Dropout desde 0.0 hasta 0.5 con un step de 0.05
        model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        
        # Creamos un bucle que permite incrementar las capas LSTM destras de la capa Dropout
        num_lstm_layers_after = hp.Int('num_lstm_layers_after', min_value=1, max_value=self.num_lstm_layers_after_input, default = self.num_lstm_layers_after_input)
        
        for i in range(num_lstm_layers_after):
            # Añadiendo capas LSTM después de Dropout
            model.add(LSTM(units=hp.Int(f'units_lstm_after_{i}', min_value=32, max_value=128, step=32),
                           return_sequences=(i < num_lstm_layers_after - 1),  # True para todas excepto la última capa LSTM
                           use_bias=True))
        
        # Capa dense que empieza con 4 unidades y va aumentando de 4 en 4 hasta 64
        model.add(Dense(hp.Int('dense_units', min_value=4, max_value=64, step=4), activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'tanh', 'elu', 'relu'],default='relu'), use_bias=True))
        
        # Se modifica la capa Dropout desde 0.0 hasta 0.5 con un step de 0.05
        model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'tanh', 'elu', 'relu'],default='relu'), use_bias=True))
        
        # Variaciones de los optimizadores
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
        if optimizer == 'adam':
            opt = Adam(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        elif optimizer == 'sgd':
            opt = SGD(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        else: # rmsprop
            opt = RMSprop(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        
        
        model.compile(optimizer=opt,
                      loss=MeanSquaredError(),
                      metrics=[
                        MeanSquaredError(name='mse'),
                        RootMeanSquaredError(name='rmse'),
                        MeanAbsolutePercentageError(name='mape')
                        ]
                    )
        
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,
            batch_size=hp.Choice("batch_size", [16, 24, 32]),
            **kwargs)

In [66]:
# def adjustHyperparameters(path, WINDOWS_SIZE=3):
WINDOWS_SIZE=3

# Leemos los archivos previamente creados
for archivo in os.listdir("RCPMergedTransferMerged"):
    
    # Comprobamos si "train" esta en el nombre del archivo
    if "train" in archivo:

        # Obtenemos el nombre del archivo
        nombreArchivo = archivo.split("_train")[0]

        # Cargamos los datos
        data_train = np.load(f'RCPMergedTransferMerged/{archivo.split("_train")[0]}_train.npz', allow_pickle=True)
        data_val = np.load(f'RCPMergedTransferMerged/{archivo.split("_train")[0]}_val.npz', allow_pickle=True)
        data_test = np.load(f'RCPMergedTransferMerged/{archivo.split("_train")[0]}_test.npz', allow_pickle=True)
        
        # Separamos segun X e y
        X_train = data_train['X_train']
        y_train = data_train['y_train']
        X_val = data_val['X_val']
        y_val = data_val['y_val']
        X_test = data_test['X_test']
        y_test = data_test['y_test']

        print(X_train.shape, X_val.shape, X_train.shape)


        # Creación de modelo de ajuste de hiperparámetros
        hypermodel = LSTMHyperModel(input_shape=(WINDOWS_SIZE+1, X_train.shape[2]), num_lstm_layers_input=3, num_lstm_layers_after_input=3)

        tuner = Hyperband(
                        hypermodel,
                        objective='val_loss',
                        max_epochs=50,
                        factor=3,
                        directory='best_models_22_10_24/LSTM',
                        project_name=f'hyperparameter_tuning_{os.path.splitext(nombreArchivo)[0]}'
                    )

        tuner.search(
            X_train, y_train,
            epochs=50,
            validation_data=(X_val, y_val)
        )
        top_models_data = []

        best_trials = tuner.oracle.get_best_trials(num_trials=10)

        for i, trial in enumerate(best_trials):
            # Obtener los hiperparámetros
            hyperparameters = trial.hyperparameters.values
            
            # Construir el modelo con los hiperparámetros del trial
            model = tuner.hypermodel.build(trial.hyperparameters)

            # Cargar los pesos del modelo desde el checkpoint
            model.load_weights(tuner.get_trial_dir(trial.trial_id) + "/checkpoint.weights.h5")

            # Evaluar el modelo en el conjunto de prueba (sin r2)
            test_loss, test_mse, test_rmse, test_mape = model.evaluate(X_test, y_test, verbose=0)
            # Eliminamos test_r2

            # Guardar el modelo entrenado
            model.save(f"RevisionPaper27-09-24/LSTM/{os.path.splitext(nombreArchivo)[0]}_model_{i+1}.h5")

            # Calcular las métricas con datos desnormalizados para los conjuntos de entrenamiento, validación y prueba

            # Entrenamiento
            predictions_train = predictionForIndividuals(X_train, y_train, model, hyperparameters["batch_size"])
            predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
            predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

            train_mse = mean_squared_error(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])
            train_rmse = np.sqrt(train_mse)
            train_mape = mean_absolute_percentage_error(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"]) *100
            train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

            # Validación
            predictions_val = predictionForIndividuals(X_val, y_val, model, hyperparameters["batch_size"])
            predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
            predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

            val_mse = mean_squared_error(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])
            val_rmse = np.sqrt(val_mse)
            val_mape = mean_absolute_percentage_error(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"]) *100
            val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

            # Prueba
            predictions_test = predictionForIndividuals(X_test, y_test, model, hyperparameters["batch_size"])
            predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
            predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

            test_mse = mean_squared_error(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])
            test_rmse = np.sqrt(test_mse)
            test_mape = mean_absolute_percentage_error(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"]) *100
            test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

            # Almacenar la información del modelo
            model_info = {
                'Model Rank': i+1,
                'Trial ID': trial.trial_id,
                # Métricas del conjunto de entrenamiento
                'train_mse': train_mse,
                'train_rmse': train_rmse,
                'train_mape': train_mape,
                'train_r2': train_r2,
                # Métricas del conjunto de validación
                'val_mse': val_mse,
                'val_rmse': val_rmse,
                'val_mape': val_mape,
                'val_r2': val_r2,
                # Métricas del conjunto de prueba
                'test_mse': test_mse,
                'test_rmse': test_rmse,
                'test_mape': test_mape,
                'test_r2': test_r2
            }

            print(f"VALORES DE R2: {test_r2}")
            # Añadir los hiperparámetros al diccionario
            model_info.update(hyperparameters)
            top_models_data.append(model_info)

        # Cargar el archivo JSON existente
        with open(f"models/LSTMMerged_22_10_24/{nombreArchivo}_best_models.json", 'w') as file:
            json.dump(top_models_data, file, indent=4)

(146905, 4, 43) (41399, 4, 43) (146905, 4, 43)

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_lstm_layers
96                |96                |units_lstm_0
32                |32                |units_lstm_1
128               |128               |units_lstm_2
0.25              |0.25              |dropout
2                 |2                 |num_lstm_layers_after
64                |64                |units_lstm_after_0
64                |64                |units_lstm_after_1
128               |128               |units_lstm_after_2
56                |56                |dense_units
tanh              |tanh              |dense_activation
0.2               |0.2               |dropout_2
adam              |adam              |optimizer
0.00049632        |0.00049632        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3       

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
2229/9182 ━━━━━━━━━━━━━━━━━━━━ 51s 7ms/step - loss: 0.0530 - mape: 93.5604 - mse: 0.0530 - rmse: 0.2296

KeyboardInterrupt: 